In [1]:
import numpy as np
import matplotlib
from matplotlib import animation
from typing import Tuple, Dict, Optional, Iterable

In [3]:
import gym
from gym import spaces

In [2]:
from pprint import pprint

In [23]:
class Maze(gym.Env):
    def __init__(self, exploring_starts: bool=False, size:int = 5) -> None:
        super().__init__()
        self.exploring_starts = exploring_starts
        self.size= size
        self.state = (size - 1, size - 1)
        self.goal = (size - 1, size - 1)
        self.maze = self._createMaze(size = self.size)
        self.action_space = spaces.Discrete(4)
        self.action_space.action_meanings = {0: 'UP', 1: 'RIGHT', 2: 'DOWN', 3: "LEFT"}

        self.distances = self._compute_distances(self.goal, self.maze, self.size)
        
        self.observation_space = spaces.MultiDiscrete([size,size])

    @staticmethod
    def _compute_distances(goal: Tuple[int,int], maze: Dict[Tuple[int, int], Iterable[Tuple[int, int]]], size: int)->np.ndarray:
        distances = np.full([size, size], np.inf)
        visited  = set()
        distances[goal] = 0

        while visited != set(maze):
            sorted_dst = [(v // 5, v % 5) for v in distances.argsort(axis=None)]
            closest = next(x for x in sorted_dst if x not in visited)
            visited.add(closest)
        
            for neighbour in maze[closest]:
                distances[neighbour] = min(distances[neighbour], distances[closest] + 1)
        return distances

    def reset(self) -> Tuple[int,int]:
        if self.exploring_starts:
            while self.state == self.goal:
                self.state = tuple(self.observation_space.sample())
        else:
            self.state=(0,0)
        return self.state

    @staticmethod
    def _createMaze(size: int):
        maze = {(row,col): [(row-1, col), (row+1,col),(row,col-1),(row,col+1)]
                for row in range(size) for col in range(size)}
        left_edges = [[(row, 0),(row,-1)] for row in range(size)]
        right_edges = [[(row, size-1), (row, size)] for row in range(size)]
        upper_edges = [[(0,col),(-1,col)] for col in range(size)]
        lower_edges = [[(size-1, col), (size,col)] for col in range(size)]

        walls = [
            [(1,0),(1,1)], [(2,0),(2,1)], [(4,0),(3,0)],
            [(0,1),(1,1)], [(3,1),(4,1)], [(2,1),(2,2)],
            [(3,1),(3,2)], [(0,3),(0,4)], [(1,3),(1,4)],
            [(0,2),(1,2)], [(1,2),(1,3)], [(2,2),(2,3)], 
            [(3,3),(3,2)], [(4,3),(3,3)], [(2,3),(2,4)],
        ]

        obstacles = upper_edges + lower_edges + right_edges + left_edges + walls

        for src, dst in obstacles:
            maze[src].remove(dst)

            if dst in maze: #this part is removing the source from the destination
                maze[dst].remove(src)
        return maze
    
    def step(self, action: int) -> Tuple[Tuple[int, int], float, bool, Dict]: #returns the next state, reward, if game is done, and additional info
        reward = self.compute_reward(self.state, action)
        self.state = self._get_next_state(self.state, action)
        done = self.state == self.goal
        info = {}
        return self.state, reward, done, info

    def compute_reward(self, state: Tuple[int,int], action: int) -> float:
        next_state = self._get_next_state(state,action)


    def _get_next_state(self, state: Tuple[int, int], action: int) -> Tuple[int,int]:
        if action == 0:
            next_state = (state[0] - 1, state[1])
        elif action == 1:
            next_state = (state[0], state[1] + 1)
        elif action == 2:
            next_state = (state[0] + 1, state[1])
        elif action == 3:
            next_state = (state[0], state[1] - 1)
        else:
            raise ValueError("Incorrect Action: ",action)

        if next_state in self.maze[state]:
            return next_state
        
        return state
    
    def simulate_step(self, state: Tuple[int,int], action: int):
        reward = self.compute_reward(self.state, action)
        next_state = self._get_next_state(self.state, action)
        done = self.state == self.goal
        info = {}
        return next_state, reward, done, info
    

In [24]:
env = Maze()

[(4, 4), (4, 2), (4, 1), (4, 0), (3, 4), (3, 3), (3, 2), (3, 1), (3, 0), (2, 4), (2, 3), (4, 3), (0, 0), (2, 0), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (0, 4), (0, 3), (0, 2), (0, 1), (2, 1), (2, 2)]
[(4, 4), (3, 4), (4, 3), (4, 2), (4, 1), (4, 0), (3, 3), (3, 2), (3, 1), (3, 0), (2, 4), (2, 3), (0, 0), (2, 0), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (0, 4), (0, 3), (0, 2), (0, 1), (2, 1), (2, 2)]
[(4, 4), (3, 4), (4, 3), (3, 3), (2, 4), (4, 2), (4, 1), (4, 0), (3, 2), (3, 1), (3, 0), (2, 3), (0, 0), (2, 0), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (0, 4), (0, 3), (0, 2), (0, 1), (2, 1), (2, 2)]
[(4, 4), (3, 4), (4, 3), (4, 2), (3, 3), (2, 4), (4, 1), (4, 0), (3, 2), (3, 1), (3, 0), (2, 3), (0, 0), (2, 0), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (0, 4), (0, 3), (0, 2), (0, 1), (2, 1), (2, 2)]
[(4, 4), (3, 4), (4, 3), (4, 2), (3, 3), (2, 4), (4, 1), (3, 2), (4, 0), (3, 1), (3, 0), (2, 3), (0, 0), (2, 0), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (0, 4), (0, 3), (0, 2), (0, 1), (2, 1), (2,

In [25]:
pprint(env.maze) # basically for taking any action ,0,1,2,3 we have the next state.

{(0, 0): [(1, 0), (0, 1)],
 (0, 1): [(0, 0), (0, 2)],
 (0, 2): [(0, 1), (0, 3)],
 (0, 3): [(1, 3), (0, 2)],
 (0, 4): [(1, 4)],
 (1, 0): [(0, 0), (2, 0)],
 (1, 1): [(2, 1), (1, 2)],
 (1, 2): [(2, 2), (1, 1)],
 (1, 3): [(0, 3), (2, 3)],
 (1, 4): [(0, 4), (2, 4)],
 (2, 0): [(1, 0), (3, 0)],
 (2, 1): [(1, 1), (3, 1)],
 (2, 2): [(1, 2), (3, 2)],
 (2, 3): [(1, 3), (3, 3)],
 (2, 4): [(1, 4), (3, 4)],
 (3, 0): [(2, 0), (3, 1)],
 (3, 1): [(2, 1), (3, 0)],
 (3, 2): [(2, 2), (4, 2)],
 (3, 3): [(2, 3), (3, 4)],
 (3, 4): [(2, 4), (4, 4), (3, 3)],
 (4, 0): [(4, 1)],
 (4, 1): [(4, 0), (4, 2)],
 (4, 2): [(3, 2), (4, 1), (4, 3)],
 (4, 3): [(4, 2), (4, 4)],
 (4, 4): [(3, 4), (4, 3)]}
